Na aula de hoje veremos o conceito de realidade aumentada e suas aplicações

Podemos tambem chamar esse conceito de modelo de cameras, pois veremos e analisaremos como uma camera funciona, então veremos alguns conceitos para identificar a distancia focal de uma camera e seus tipos de lentes

Apresentaremos os conceitos agora:

1. Lente esferica
    
    A lente esferica é a mesma ideia ja conhecida de tempos anteriores chamado de lente esferica, ou seja, uma imagem refletida sobre uma lente, terá um ponto focal que os unes, o mesmo chamado de eixo otico a linha que gera o ponto focal da camera, em outras palavras, o ponto que une todas as informações da imagem, a distancia da lente para um ponto é chamado de distancia focal
    
2. Plano focal
    
    É a ideia da distancia focal da lente e um plano que receberá os pixeis de intensidade dado pela iimagem tirada, assim, esse plano sera um plano atras da lente para receber os pixeis, se for mais perto ou mais longe da lente, ele causará uma imagem borrada, pois os pixeis de intensidade serao divididos em varios pixeis armazenados, assim, mesclando objetos.
    
3. Captura da imegm de um objeto

    Como capturamos uma imagem então? Para capturar uma  imagem, nós basicamente armazenamos os feixes de luzes passados pelo objeto
    
    A ideia de camea pinhole se mantem aqui, basicamente, se o buraco de uma camera pinhole for maior, ele deixara a imagem mais borrada, pois um ponto (um feixe de luz) será maior, assim, o mesmo pixel de valor de intensidade será distribuido para mais pixeis doq ele realmente é, assim, ficaraá borrado. Se usarmos uma lente junto com essa camera, a imagem recebera mais feixes de luz com maior nitides e assim, o sensor capturará uma imagem de melhor qualidade, mas se deixarmos o plano focal mais longe, a camerá receberá uma imagem borrada pelo mesmo motivo do pinhole.
    
    
Em geral, a distancia focal de uma camera para o plano focal é diferente em X e Y, portanto:

xi = fx * $\frac {xc}{zc}$

yi = fy * $\frac {yc}{zc}$

Sendo f a distancia focal da lente para o plano focal.

Transformando isso em coordenadas homogenias, teremos:

[xi] = [fx 0 0] * [xc]

[yi] = [0 fy 0] * [yc]

[N] = [0 0 1] * [zc]

Onde N ali não existe, só deixei ali pra n ficar feio na hora de fazer a demonstração (tenho mt a aprender de markdown ainda kkkkk)

Temos que deixar essa matrix acima mais geral, pois ela ainda está mt confusa e não aborda todoos os pontos que queremos ressaltar

[xi] = [fx 0 cx] * [xc]

[yi] = [0 fy cy] * [yc]

[N] = [0 0 1] * [zc]

Novamente, esse N ai n existe

A segunda matriz é chamada de matriz intríseca da camera

A estimativa de pose de um objeto é as coordenadas de um objeto capturado pela camera, assim, sendo os pontos (xo->, yo->, zo->). A estimativa de pose é basicamente fazer a rotação e a translação do objeto em relação a camera, assim, conseguimos fazer a identificação de todos os pontos do objeto e melhor visualização do objeto

As coordenadas (xo, yo, zo) e (xc, yc, zc) estão relacionados por uma translação + rotação

Podemos então escrever a rotação do objeto para a camera:

rc-> =

[r1 r2 r3 tx] * ro->

[r4 r5 r6 ty]

[r7 r8 r9 tz]

A matrix intriseca chamaremos de Mi e extrisica Me

a matrix acima é igual a 

Me * ro->


Resumindo

ri-> = Mi * Me * ro->

ri é os pixeis da imagem, Mi a matriz intriseca, Me a matrix extrinsica e ro-> o vetor do objeto

Portanto, podemos mapear qualquer ponto na coordenada do objeto no pixel correspondente

Dado um conjunto de pontos do objeto ro-> e ri-> (as posições do objeto na imagem), minimizamos :

S = $\sum_{j=1}^{N} (ri-> - Mi*Me*ro->)²$

isso, em outras palavras é o erro quadratico da diferença da distancia do objeto para o centro da imagem

In [1]:
import cv2
import numpy as np
import os
from pathlib import Path

In [2]:
def detect_corners(data_dir, grid_size):
    files = os.listdir(data_dir)
    criteria = (cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 30, 0.1)
    img_points = []
    for fname in files:
        img = cv2.imread(str(data_dir/fname))
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(img_gray, grid_size)
        
        if ret == True:
            corners_sub = cv2.cornerSubPix(img_gray, corners, (11, 11), (-1, -1), criteria)
            img_points.append(corners_sub)
        else:
            print('Pontos não detectados na imagem:', fname)
    return img_points

def check_corners_detection(data_dir, grid_size, img_points):
    files = os.listdir(data_dir)
    for fname, corners in zip(files, img_points):
        img = cv2.imread(str(data_dir/fname))
        cv2.drawChessboardCorners(img, grid_size, corners, True)
        
        cv2.imshow('img', img)
        cv2.waitKey(0)
    
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
    return
        
grid_size = (7, 6)
dir_img = Path('../board')

In [3]:
img_points = detect_corners(dir_img, grid_size)
check_corners_detection(dir_img, grid_size, img_points)

In [4]:
def chessboard_grid_points(grid_size):
    obj_points = np.zeros((grid_size[0]*grid_size[1], 3), dtype = np.float32)
    obj_points[:, 0:2] = np.mgrid[0:grid_size[0], 0:grid_size[1]].transpose().reshape(-1, 2)
    
    return obj_points

def calibrate(obj_points, img_points, img_size):
    flags = cv2.CALIB_ZERO_TANGENT_DIST + cv2.CALIB_FIX_K1 +\
    cv2.CALIB_FIX_K2 + cv2.CALIB_FIX_K3
    
    error, mat_int, dist, rvecs, tvects =cv2.calibrateCamera(obj_points, img_points, img_size, 
                        cameraMatrix = None, distCoeffs = None, flags = flags)
    
    return mat_int, error

img = cv2.imread(str(dir_img/'imagem1.jpg'))
obj_p = chessboard_grid_points(grid_size)
obj_points = []
for i in range(len(img_points)):
    obj_points.append(obj_p)
    
mat_int, error = calibrate(obj_points, img_points, img.shape[:-1][::-1])
print(mat_int)
print(f'fx = {mat_int[0,0]}, fy = {mat_int[1,1]}, cx = {mat_int[0,2]}, cy = {mat_int[1,2]}')

[[1.49704831e+03 0.00000000e+00 9.05467318e+02]
 [0.00000000e+00 1.54857470e+03 4.97768942e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
fx = 1497.048313452922, fy = 1548.574703950075, cx = 905.4673181973168, cy = 497.76894215427814


In [13]:
def drawn_axes(img, proj_points):
        img = cv2.line(img, pt1 = proj_points[0], pt2 = proj_points[1], 
                       color = (255, 0, 0), thickness = 5)
        img = cv2.line(img, pt1 = proj_points[0], pt2 = proj_points[2], 
                       color = (0, 255, 0), thickness = 5)
        img = cv2.line(img, pt1 = proj_points[0], pt2 = proj_points[3], 
                       color = (0, 0, 255), thickness = 5)
        return img

def drawn_axes_on_img(data_dir, mat_int, obj_points, img_points):
    axes = np.array([[0,0,0], 
                     [3, 0, 0],
                     [0, 3, 0],
                     [0, 0, -3]], np.float32)
    
    files = os.listdir(data_dir)
    for idx, fname in enumerate(files):
        img = cv2.imread(str(data_dir/fname))
        ret, rvec, tvec = cv2.solvePnP(obj_points[idx], img_points[idx], mat_int, None)
        if ret == True:
            proj_points, _ = cv2.projectPoints(axes, rvec, tvec, mat_int, None)
            
            #arredondando e removendo uma dimensão inutil
            proj_points = np.round(proj_points).astype(int).squeeze()
            
            img = drawn_axes(img, proj_points)
            cv2.imshow('img', img)
            cv2.waitKey(0)
        else:
            print(f'nao foi possivel estimar para a imagem {img}')
            
    cv2.destroyAllWindows()
    cv2.waitKey(2)

In [14]:
drawn_axes_on_img(dir_img, mat_int, obj_points, img_points)